# MoRT and Distress / Empathy dataset
In this script, I will do a correlation analysis of the moral dimension (MD) of Schramowski et al. 2022 and the empathy dataset.

Schramowski et al. 2022  
"Large Pre-trained Language Models Contain Human-like Biases of What is Right and Wrong to Do" 



### Run
run moral_empathy_anaylsis.py before to create the output in output/moral_output/

In [1]:
import pickle
import pandas as pd
import sklearn
print (sklearn.__version__)
from sklearn.decomposition import PCA
from matplotlib import style 
import seaborn as sns
style.use('default')
import matplotlib.pyplot as plt
import numpy as np
from random import random
import decimal
import math
output_dir = "../output/moral_output/"
COLORS = ['#029e72', '#e69f00', '#f0e441', '#57b4e8']

1.1.1


## Load moral dimension

In [2]:

def bin_data(labels, moral_pca, bin_size=0.1, return_bins=False):
    # - create bins -
    decimal_count = abs(decimal.Decimal(str(bin_size)).as_tuple().exponent)
    min_score = min([item for item in labels])
    max_score = max([item for item in labels])
    bins_start = math.floor(min_score * (10**decimal_count)) / (10**decimal_count)
    bins_end = math.ceil(max_score * (10**decimal_count)) / (10**decimal_count)
    # add the end point to the bins as well, to get the upper range for the elements
    # this will be removed later on, since it is not actually a bin
    bins = np.arange(bins_start, bins_end + bin_size, bin_size)

    # - divide data into bins - 
    binned_pca = [[] for i in range(len(bins))]
    binned_labels = [[] for i in range(len(bins))]
    for idx, score in enumerate(labels):
        min_idx = np.where(bins <= score)[0]
        max_idx = np.where(bins > score)[0] - 1
        item_bin_idx = np.intersect1d(min_idx, max_idx)[0]
        moral_pca_i = moral_pca[idx]
        binned_pca[item_bin_idx].append(moral_pca_i)
        binned_labels[item_bin_idx].append(score)
    # remove last bin, because it is 0 anyways, just needed it for the calculation
    binned_pca = binned_pca[:-1]
    binned_labels = binned_labels[:-1]
    bins = bins[:-1]

    return binned_pca, binned_labels, bins
        

#binned_pca, binned_labels, bins = bin_data(labels, moral_pca, 0.1, True)
#print(bins)
#print(binned_pca)
#print(binned_labels)


In [3]:


def plot_moral_empdis(bins, binned_data):
    colors = ['red', 'blue', 'yellow']
    binned_ave, binned_std, final_bins = [], [], []
    for idx, bin in enumerate(binned_data):
        if len(bin) >= 1:
            ave_bin = np.mean(bin, axis=0)
            std_bin = np.std(bin, axis=0)
            binned_ave.append(ave_bin)
            binned_std.append(std_bin)
            final_bins.append(bins[idx])
            
    binned_ave = np.array(binned_ave)
    binned_std = np.array(binned_std)

    lower_std_bound = binned_ave - binned_std
    upper_std_bound = binned_ave + binned_std
    for i in range(binned_ave.shape[1]):
        plt.plot(final_bins, binned_ave[:, i], c=COLORS[i], label=f'PC {i+1}')
        plt.fill_between(final_bins, lower_std_bound[:, i], upper_std_bound[:, i], color=COLORS[i], alpha=0.5)
    plt.title('The average moral score with the std')
    plt.xlabel('Average moral score')
    plt.ylabel('Score (in bins)')
    plt.savefig(output_dir)
    return binned_ave, binned_std, final_bins


In [4]:

def plot_distance_moral_empdis(bins, binned_pca, binned_labels):

    for idx, bin in enumerate(binned_pca):
        if len(bin) >= 1:
            print('bin', bin)
            print('binned_labels', binned_labels[idx])
            distance = np.array(bin) - np.array(binned_labels[idx])
            print('distance', distance)
            ave_bin = np.mean(bin, axis=0)
            std_bin = np.std(bin, axis=0)
            print(ave_bin)
            print(std_bin)
            print()


In [5]:
def outlier_detection(data_array):
    kept_idx, new_data = [], []
    avr = np.mean(data_array)
    std = np.std(data_array)
    is_outlier = lambda x: item > avr + std*2 or item < avr - std*2
    outlier_count = 0

    for idx, item in enumerate(data_array):
        if is_outlier(item):
            outlier_count += 1
            print('outlier:', item)
            continue
        new_data.append(item)
        kept_idx.append(idx)
    kept_idx = np.array(kept_idx)
    new_data = np.array(new_data)
    print(f'Detected {outlier_count} outlier(s)')
    return new_data, kept_idx


In [6]:
md_results_old = pd.read_csv('../output/moral_output/moral_correlations_old.csv')

In [7]:
md_results = pd.read_csv('../output/moral_output/moral_correlations.csv')
md_results["pearson_p"] = md_results["pearson_p"].round(6)
md_results

,Unnamed: 0,pearson_r,pearson_p,princ_comp,note,task_name
0,0,-0.05079915121919156,0.028467,1,MD ess - label (With outliers),empathy
1,1,-0.0314310586370807,0.175426,2,MD ess - label (With outliers),empathy
2,2,0.18179710076857938,0.000000,3,MD ess - label (With outliers),empathy
3,3,-0.021514262519644507,0.353748,4,MD ess - label (With outliers),empathy
4,4,-0.27918470540538204,0.000000,5,MD ess - label (With outliers),empathy
5,5,[-0.0007430014976317328],0.974454,1,MoRT_art - labels,empathy
6,6,0.39613888480129283,0.000000,1,MoRT_art - MoRT_essay,empathy
7,7,[0.01989453968063049],0.391161,1,"Sim(MoRT_art, MoRT_essay) - labels",empathy
8,8,[0.08816221652999558],0.000141,2,MoRT_art - labels,empathy
9,9,0.30533943097880245,0.000000,2,MoRT_art - MoRT_essay,empathy


In [18]:
task_name = 'distress'
md_results[(md_results.task_name == task_name) & (md_results.note =='Sim(MoRT_art, MoRT_essay) - labels')]

,Unnamed: 0,pearson_r,pearson_p,princ_comp,note,task_name
27,27,[-0.037814050179319915],0.103034,1,"Sim(MoRT_art, MoRT_essay) - labels",distress
30,30,[-0.016438120893688565],0.478629,2,"Sim(MoRT_art, MoRT_essay) - labels",distress
33,33,[-0.049579945335238025],0.032505,3,"Sim(MoRT_art, MoRT_essay) - labels",distress
36,36,[0.004842601114233688],0.834674,4,"Sim(MoRT_art, MoRT_essay) - labels",distress
39,39,[-0.09493373094005567],0.000041,5,"Sim(MoRT_art, MoRT_essay) - labels",distress


In [17]:
sim_results = md_results[(md_results.note =='Sim(MoRT_art, MoRT_essay) - labels')]
sim_results['pearson_r']

7       [0.01989453968063049]
10     [-0.07201184985357337]
13      [-0.0795929982617105]
16     [-0.05717668055905381]
19     [-0.06552986876810704]
27    [-0.037814050179319915]
30    [-0.016438120893688565]
33    [-0.049579945335238025]
36     [0.004842601114233688]
39     [-0.09493373094005567]
Name: pearson_r, dtype: object